In [176]:
import pandas as pd

In [382]:
df = pd.read_csv('Nba_player_data_2013.csv')

In [383]:
df.head()

,Unnamed: 0,+/1,3P,3PA,AST,BLK,Date,FG,FGA,FT,...,TOV,TRB,Team,minutes,name,opp_score,started,team_score,is_Home,Team_Win
0,0,-14.0,1.0,5.0,1.0,0.0,10/29/2013,3.0,14.0,2.0,...,1.0,3.0,Orlando,32:59,Arron Afflalo,97,1,87,0,0
1,1,-22.0,0.0,0.0,3.0,1.0,10/29/2013,4.0,11.0,0.0,...,5.0,10.0,Orlando,30:39,Nikola Vucevic,97,1,87,0,0
2,2,-11.0,3.0,7.0,7.0,0.0,10/29/2013,4.0,13.0,1.0,...,2.0,5.0,Orlando,30:34,Jameer Nelson,97,1,87,0,0
3,3,-15.0,0.0,0.0,0.0,2.0,10/29/2013,0.0,5.0,0.0,...,2.0,5.0,Orlando,26:19,Jason Maxiell,97,1,87,0,0
4,4,-3.0,2.0,2.0,0.0,0.0,10/29/2013,6.0,13.0,0.0,...,0.0,1.0,Orlando,23:30,Maurice Harkless,97,1,87,0,0


In [384]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [385]:
df.head()

,+/1,3P,3PA,AST,BLK,Date,FG,FGA,FT,FTA,...,TOV,TRB,Team,minutes,name,opp_score,started,team_score,is_Home,Team_Win
0,-14.0,1.0,5.0,1.0,0.0,10/29/2013,3.0,14.0,2.0,3.0,...,1.0,3.0,Orlando,32:59,Arron Afflalo,97,1,87,0,0
1,-22.0,0.0,0.0,3.0,1.0,10/29/2013,4.0,11.0,0.0,0.0,...,5.0,10.0,Orlando,30:39,Nikola Vucevic,97,1,87,0,0
2,-11.0,3.0,7.0,7.0,0.0,10/29/2013,4.0,13.0,1.0,1.0,...,2.0,5.0,Orlando,30:34,Jameer Nelson,97,1,87,0,0
3,-15.0,0.0,0.0,0.0,2.0,10/29/2013,0.0,5.0,0.0,0.0,...,2.0,5.0,Orlando,26:19,Jason Maxiell,97,1,87,0,0
4,-3.0,2.0,2.0,0.0,0.0,10/29/2013,6.0,13.0,0.0,1.0,...,0.0,1.0,Orlando,23:30,Maurice Harkless,97,1,87,0,0


In [386]:
df['spread'] = df['team_score']-df['opp_score']

In [387]:
df.columns

Index([u'+/1', u'3P', u'3PA', u'AST', u'BLK', u'Date', u'FG', u'FGA', u'FT',
       u'FTA', u'Game_Id', u'ORB', u'Opponent', u'PF', u'PTS', u'STL', u'TOV',
       u'TRB', u'Team', u'minutes', u'name', u'opp_score', u'started',
       u'team_score', u'is_Home', u'Team_Win', u'spread'],
      dtype='object')

In [388]:
groups= df.groupby(['Game_Id','Team'],sort=False).sum()

In [389]:
groups.head()

+/1    3P   3PA   AST   BLK  \
Game_Id                          Team                                        
10/29/2013_Indiana_Orlando       Orlando     -50.0   9.0  19.0  17.0   6.0   
                                 Indiana      50.0   7.0  17.0  17.0  18.0   
10/29/2013_LA Lakers_LA Clippers LA Clippers -65.0   8.0  21.0  27.0   4.0   
                                 LA Lakers    65.0  14.0  29.0  23.0   6.0   
10/29/2013_Miami_Chicago         Chicago     -60.0   7.0  26.0  23.0   4.0   

                                                FG   FGA    FT   FTA   ORB  \
Game_Id                          Team                                        
10/29/2013_Indiana_Orlando       Orlando      36.0  93.0   6.0  10.0  13.0   
                                 Indiana      34.0  71.0  22.0  32.0  10.0   
10/29/2013_LA Lakers_LA Clippers LA Clippers  41.0  83.0  13.0  23.0  10.0   
                                 LA Lakers    42.0  93.0  18.0  28.0  18.0   
10/29/2013_Miami_Chicago         Chicago      35.0  83.0  18.0  23.0  11.0   

                                               ...      PTS   STL   TOV   TRB  \
Game_Id                          Team          ...                              
10/29/2013_Indiana_Orlando       Orlando       ...     87.0  10.0  17.0  39.0   
                                 Indiana       ...     97.0   4.0  20.0  44.0   
10/29/2013_LA Lakers_LA Clippers LA Clippers   ...    103.0  11.0  16.0  40.0   
                                 LA Lakers     ...    116.0   8.0  19.0  52.0   
10/29/2013_Miami_Chicago         Chicago       ...     95.0  11.0  18.0  41.0   

                                              opp_score  started  team_score  \
Game_Id                          Team                                          
10/29/2013_Indiana_Orlando       Orlando           1261        5        1131   
                                 Indiana           1131        5        1261   
10/29/2013_LA Lakers_LA Clippers LA Clippers       1508        5        1339   
                                 LA Lakers         1339        5        1508   
10/29/2013_Miami_Chicago         Chicago           1391        5        1235   

                                              is_Home  Team_Win  spread  
Game_Id                          Team                                    
10/29/2013_Indiana_Orlando       Orlando            0         0    -130  
                                 Indiana           13        13     130  
10/29/2013_LA Lakers_LA Clippers LA Clippers        0         0    -169  
                                 LA Lakers         13        13     169  
10/29/2013_Miami_Chicago         Chicago            0         0    -156  

[5 rows x 21 columns]

In [390]:
matrix = groups.as_matrix()

In [391]:
ids = []
for game in df['Game_Id'].unique():
    ids.append(game+ '%aa'+game.split('_')[1])
    ids.append(game+ '%aa'+game.split('_')[2])
ids[0:10]

['10/29/2013_Indiana_Orlando%aaIndiana',
 '10/29/2013_Indiana_Orlando%aaOrlando',
 '10/29/2013_LA Lakers_LA Clippers%aaLA Lakers',
 '10/29/2013_LA Lakers_LA Clippers%aaLA Clippers',
 '10/29/2013_Miami_Chicago%aaMiami',
 '10/29/2013_Miami_Chicago%aaChicago',
 '10/30/2013_Cleveland_Brooklyn%aaCleveland',
 '10/30/2013_Cleveland_Brooklyn%aaBrooklyn',
 '10/30/2013_Dallas_Atlanta%aaDallas',
 '10/30/2013_Dallas_Atlanta%aaAtlanta']

In [392]:
len(matrix)

2538

In [368]:
new_frame = pd.DataFrame(matrix, columns = groups.columns, index = ids)

In [369]:
team_data = new_frame[['3P','3PA', 'AST', 'BLK', 'FG', 'FGA', 'FT', 'FTA', 'ORB',
       'PF', 'PTS', 'STL', 'TOV', 'TRB','is_Home','Team_Win']]

In [370]:
team_data = team_data.reset_index()

In [371]:
team_data['Team'] = team_data['index'].apply(lambda x: x.split('%aa')[1])

In [372]:
team_data['index'] = team_data['index'].apply(lambda x: x.split('%aa')[0])

In [373]:
team_data['is_Home'] = team_data['is_Home'].apply(lambda x: x>0).astype(int)
team_data['Team_Win'] = team_data['Team_Win'].apply(lambda x: x>0).astype(int)

In [374]:
team_data.head()

,index,3P,3PA,AST,BLK,FG,FGA,FT,FTA,ORB,PF,PTS,STL,TOV,TRB,is_Home,Team_Win,Team
0,10/28/2014_LA Lakers_Houston,12.0,29.0,22.0,3.0,31.0,73.0,34.0,50.0,14.0,30.0,108.0,7.0,13.0,47.0,0,1,LA Lakers
1,10/28/2014_LA Lakers_Houston,3.0,10.0,16.0,3.0,28.0,79.0,31.0,39.0,11.0,32.0,90.0,7.0,11.0,36.0,1,0,Houston
2,10/28/2014_New Orleans_Orlando,4.0,11.0,17.0,9.0,32.0,84.0,16.0,21.0,16.0,25.0,84.0,5.0,18.0,56.0,0,0,New Orleans
3,10/28/2014_New Orleans_Orlando,4.0,17.0,20.0,17.0,41.0,101.0,15.0,31.0,26.0,17.0,101.0,10.0,9.0,62.0,1,1,Orlando
4,10/28/2014_San Antonio_Dallas,8.0,21.0,17.0,3.0,38.0,78.0,16.0,19.0,9.0,20.0,100.0,9.0,10.0,33.0,0,0,San Antonio


In [375]:
both_teams_total = team_data.groupby('index',sort=False).sum().reset_index()
##doing weird things. 
weird = pd.merge(both_teams_total,team_data,on='index')

In [376]:
#fill in opponent data in an insufficient way
team_data['opp_3P'] = weird['3P_x']-team_data['3P']
team_data['opp_3PA'] = weird['3PA_x']-team_data['3PA']
team_data['opp_AST'] = weird['AST_x']-team_data['AST']
team_data['opp_BLK'] = weird['BLK_x']-team_data['BLK']
team_data['opp_FG'] = weird['FG_x']-team_data['FG']
team_data['opp_FGA'] = weird['FGA_x']-team_data['FGA']
team_data['opp_FT'] = weird['FT_x']-team_data['FT']
team_data['opp_FTA'] = weird['FTA_x']-team_data['FTA']
team_data['opp_PTS'] = weird['PTS_x']-team_data['PTS']
team_data['opp_ORB'] = weird['ORB_x']-team_data['ORB']
team_data['opp_STL'] = weird['STL_x']-team_data['STL']
team_data['opp_PF'] = weird['PF_x']-team_data['PF']
team_data['opp_TOV'] = weird['TOV_x']-team_data['TOV']

In [377]:
def opponent(row):
    if row['Team'] == row['index'].split('_')[1]:
        return row['index'].split('_')[2]
    else:
        return row['index'].split('_')[1]

In [378]:
team_data['opponent'] =team_data.apply(opponent, axis=1)

In [379]:
team_data['Game_Id'] = team_data['index']
team_data.drop('index',axis=1, inplace=True)

In [380]:
team_data.shape

(2460, 32)

In [381]:
##get rid of games that are playoffs

team_data.to_csv('team_boxscores_2014.csv')